In [32]:
%reload_ext autoreload
%autoreload 2


# Week 5: Systematically Improving Your RAG Application

> **Note:** Please walk through the previous 2 notebooks 1. Generate Dataset.ipynb  and 2. Metadata Filtering.ipynb before continuing with this notebook. If you haven't generated the dataset, please run `1. Generate Dataset.ipynb` first.

# Why Text-2-SQL?

Models can generate SQL in response to user queries. This allows our model to be able to interact with an external database and answer a greater variety of questions. This is known as Text-2-SQL, where a model takes an intial bit of user text and outputs a SQL query.

We want to implement some form of text-2-sql because it is an easy way to provide more complex functionality before we invest time into building dedicated infrastructure for specific use-cases.

In this notebook, we'll implement the basic text-2-sql capabilities with the aid of a local sqlite database. We'll do so in 3 steps

1. **Synthetic Data** : First, we'll generate some synthetic users and order data and populate a local sqlite database with that data to allow us to run queries against it.
2. **Generating SQL** : Next, we'll show how we can use function calling to generate valid SQL queries.
3. **Integrating it into our RAG pipeline** : Finally, we'll show how we can integrate it into our existing RAG pipeline, combing query understanding with a text-2-sql function call to generate a response to a user query.

# Creating a Synthetic Database

We want to create a synthetic dataset that we can run simple queries against. 

We use `faker` here because it allows us to generate realistic looking data. We'll also be using our original `taxonomy.yml` file to ensure that the generated data conforms to our size and color taxonomy definitions. While we're using `sqlite3` here for our database, you can use any other database you want.

Lastly, we'll also be using the `labelled-ecommerce-taxonomy` dataset to ensure that the `product_id`s in our database are valid. 

## Initialising The Database

We'll start by initialising a database and populating it with some fake data. The database will have the following tables

- `stock`: Contains information on specific items, including their color, size, and available quantity.
- `orders`: Stores order details, including user email, order date, and total amount.
- `sales`: Records sales transactions, linking each sale to an order and detailing the items sold.
- `users`: Represents users with accounts in our store, storing their email, name, gender, and other personal details.

We take a few steps here to try and make our data more realistic.

We use a `UserConfig` model here to store simulated user preferences - specifically we want to make sure that users always choose a consistent set of colors and buy sizes that are within a certain range. This means that they can always choose a consistent set of colors and sizes which would mimic a real user.

We also generate and vary the availability of items in our database to simulate how a real e-commerce store would have different stock levels for items

Let's see how this works in practice.

In [35]:
from helpers import process_taxonomy_file

taxanomy_data = process_taxonomy_file("./taxonomy.yml")
colors = taxanomy_data["common_attributes"]["Color"]
sizes = taxanomy_data["common_attributes"]["Size"]

In [34]:
import random
from faker import Faker

# Seed Faker for consistent results
Faker.seed(42)
random.seed(42)

fake = Faker()
fake.unique.clear()


def generate_size_preference(sizes):
    idx = random.choice([i for i in range(len(sizes))])
    size_pref = random.choice([1, 2])
    if idx == 0:
        return sizes[:2] if size_pref == 2 else [sizes[0]]

    if idx == len(sizes) - 1:
        return sizes[-2:] if size_pref == 2 else sizes[-1:]

    return sizes[idx - 1 : idx + 1] if size_pref == 2 else [sizes[idx]]


def generate_color_preference(colors):
    return random.sample(colors, k=random.randint(1, len(colors)))

In [3]:
from pydantic import BaseModel
from typing import Literal
from datetime import datetime
from rich import print
import random


def generate_email_from_name(name: str) -> str:
    # Convert name to lowercase and split into parts
    name_parts = name.lower().split()
    first_name = name_parts[0]
    last_name = name_parts[-1] if len(name_parts) > 1 else ""

    # Different email patterns
    patterns = [
        f"{first_name}_{last_name}",  # lisa_burns
        f"{first_name[0]}_{last_name}",  # l_burns
        f"{first_name}.{last_name}",  # lisa.burns
        f"{first_name}{last_name}",  # lisaburns
        f"{first_name[0]}{last_name}",  # lburns
    ]

    domains = [
        "gmail.com",
        "yahoo.com",
        "hotmail.com",
        "outlook.com",
        "icloud.com",
    ]

    email = f"{random.choice(patterns)}@{random.choice(domains)}"
    return email


class UserConfig(BaseModel):
    name: str
    email: str
    gender: Literal["M", "F"]
    date_of_birth: str
    color_pref: list[str]
    size_pref: list[str]
    created_at: str


def generate_user() -> UserConfig:
    name = fake.unique.name_female()
    return UserConfig(
        name=name,
        email=generate_email_from_name(name),
        gender="F",
        date_of_birth=fake.date_of_birth(minimum_age=20, maximum_age=40).strftime(
            "%Y-%m-%d"
        ),
        color_pref=generate_color_preference(colors),
        size_pref=generate_size_preference(sizes),
        created_at=fake.unique.date_between(
            start_date=datetime(2023, 1, 1), end_date=datetime(2024, 12, 31)
        ).strftime("%Y-%m-%d"),
    )


print(generate_user())

UserConfig(
    name='Allison Hill',
    email='allison_hill@gmail.com',
    gender='F',
    date_of_birth='1988-08-14',
    color_pref=[
        'Blue',
        'Navy',
        'Brown',
        'Gray',
        'White',
        'Red',
        'Green',
        'Beige',
        'Black',
        'Purple',
        'Yellow',
        'Pink'
    ],
    size_pref=['XXS'],
    created_at='2024-06-21'
)

In [4]:
from pydantic import Field


class ItemData(BaseModel):
    product_id: int
    price: float


class SoldItem(BaseModel):
    product_id: int
    size: str
    color: str
    quantity: int
    price: float


class UserOrder(BaseModel):
    user_email: str
    items: list[SoldItem]
    date: str = Field(
        default_factory=lambda: fake.date_between(
            start_date=datetime(2023, 1, 1), end_date=datetime(2023, 12, 31)
        ).strftime("%Y-%m-%d")
    )
    total_amount: float

In [5]:
def generate_random_order(
    items: list[ItemData],
    users: list[UserConfig],
):
    order_item_count = random.randint(1, 5)
    user = random.choice(users)
    order_items = random.sample(items, order_item_count)

    user_sizes = user.size_pref
    user_colors = user.color_pref

    orders = []
    total = 0
    for item in order_items:
        orders.append(
            SoldItem(
                product_id=item.product_id,
                size=random.choice(user_sizes),
                color=random.choice(user_colors),
                price=item.price,
                quantity=random.choice(
                    [1, 2]
                ),  # User buys at most 2 of each item in a single order,
            )
        )
        total += item.price * orders[-1].quantity

    return UserOrder(user_email=user.email, items=orders, total_amount=total)

In [8]:
from datasets import load_dataset

NUMBER_USERS = 100
NUMBER_ORDERS = 200

users = [generate_user() for _ in range(NUMBER_USERS)]

items = [
    ItemData(product_id=item["id"], price=item["price"])
    for item in load_dataset("ivanleomk/labelled-ecommerce-taxonomy")["train"]
]

user_emails = set([user.email for user in users])
len(user_emails)

100

In [9]:
orders = [generate_random_order(items, users) for _ in range(NUMBER_ORDERS)]

In [10]:
from itertools import product
from helpers import process_taxonomy_file
import pandas as pd


def generate_combinations(taxonomy_data: dict):
    """Generate a random subset of combinations of colors and sizes from the taxonomy"""
    import random

    # Get colors and sizes from taxonomy data
    common_attrs = taxonomy_data["common_attributes"]

    colors = common_attrs["Color"]
    sizes = common_attrs["Size"]

    # Select random number of colors and sizes
    num_colors = random.randint(1, len(colors))
    num_sizes = random.randint(1, len(sizes))

    # Randomly sample colors and sizes
    selected_colors = random.sample(colors, num_colors)
    selected_sizes = random.sample(sizes, num_sizes)

    # Generate random stock count for each color-size combination
    stock_dict = {}
    for color, size in product(selected_colors, selected_sizes):
        stock_dict[(color, size)] = random.randint(1, 30)
    return stock_dict


taxonomy_data = process_taxonomy_file("./taxonomy.yml")
combinations = []
for item in load_dataset("ivanleomk/labelled-ecommerce-taxonomy")["train"]:
    stock_dict = generate_combinations(taxonomy_data)
    for (color, size), stock in stock_dict.items():
        combinations.append(
            {"product_id": item["id"], "color": color, "size": size, "stock": stock}
        )

combinations = pd.DataFrame(combinations)
combinations

,product_id,color,size,stock
0,1,Pink,XS,1
1,1,Pink,M,15
2,1,Pink,S,9
3,1,Pink,XXXL,7
4,1,Pink,XL,25
...,...,...,...,...
5763,191,Navy,XXXL,23
5764,191,Navy,XXS,25
5765,191,Blue,S,8
5766,191,Blue,XXXL,22


## Populating the Database

Now that we've generated some initial fake data to represent item stock and availability, we'll insert it into our database. 

In [11]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("./db.sqlite")
cursor = conn.cursor()

# Create tables from init.sql
with open("init.sql", "r") as sql_file:
    init_sql = sql_file.read()
    cursor.executescript(init_sql)

In [12]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("./db.sqlite")
cursor = conn.cursor()

# Insert users data
for user in users:
    cursor.execute(
        """
        INSERT INTO users (email, name, gender, date_of_birth, created_at)
        VALUES (?, ?, ?, ?, ?)
    """,
        (user.email, user.name, user.gender, user.date_of_birth, user.created_at),
    )

# Insert orders and sales data
for order in orders:
    # Insert order
    cursor.execute(
        """
        INSERT INTO orders (user_email, date, total_amount)
        VALUES (?, ?, ?)
    """,
        (order.user_email, order.date, order.total_amount),
    )

    order_id = cursor.lastrowid  # Get the ID of the last inserted order

    # Insert sales for each item in the order
    for item in order.items:
        cursor.execute(
            """
            INSERT INTO sales (order_id, product_id, size, color, quantity, price_per_unit)
            VALUES (?, ?, ?, ?, ?, ?)
        """,
            (
                order_id,
                item.product_id,
                item.size,
                item.color,
                item.quantity,
                item.price,
            ),
        )

# Insert stock data from combinations DataFrame
for _, row in combinations.iterrows():
    cursor.execute(
        """
        INSERT INTO stock (product_id, color, size, quantity)
        VALUES (?, ?, ?, ?)
    """,
        (row["product_id"], row["color"], row["size"], row["stock"]),
    )

# Commit the changes and close the connection
conn.commit()
conn.close()

# Implementing Text-2-SQL

We can use Function Calling to allow our model to write SQL queries. This is useful because we can abstract away the complexity of SQL into a simple function call.

We'll do so in 3 steps

1. First we'll show how we can use function calling to generate valid SQL
2. Then we'll show how we can allow the model to decide whether it needs to make a SQL call or not with `instructor` using an `optional` field.
3. Lastly, we'll combine it with our previous flow to allow the model to write SQL when it gets provided with a user query and a set of relevant items we've fetched from our catalogue

## Generating SQL

We can use function calling to generate valid SQL. However, in order to ensure our model can generate SQL that's specific to the user's needs. 

We'll do two things

Firstly, we'll give our model a `read-only` connection as the default. This ensures that we don't run any queries that might potentially alter the database.

Secondly, we'll provide context to our model so that it can generate accurate and valid SQL. In our case, this information is

1. User Specific - we provide the user's email and name so that the model can generate SQL that's specific to the user. 

2. Database Schema - we provide the database schema so that the model can generate SQL that's valid given the specific database schema that we have in mind

We also provide some simple rules as you'll see later such as to use the item_id as the product id when writing sql tables and that price is not a field to worry about since we've already filtered on price.

In your own application, depending on the specific use case, you might require even more information such as user preference, past orders etc.

Let's see how this works in practice

In [39]:
import sqlite3

# Connect to the database with a read only connection
conn = sqlite3.connect("file:./db.sqlite?mode=ro", uri=True)
cursor = conn.cursor()

cursor.execute("SELECT * FROM users LIMIT 1")

# Fetch the user's email and name
email, name, *rest = cursor.fetchall()[0]
print(email, name)

allison_hill@gmail.com Allison Hill

By using a read only connection, we've ensured that we can't run any queries that might potentially alter the database. This limits the potential damage that running the model's generated SQL might cause.

In production, some other common mitigations we can do are

1. Providing the model sandboxed access to specific bits of information - using mechanisms such as Row Level Security in postgres
2. Having a limited set of SQL commands ( making the model generate values that we can then plug into placeholder strings )

We've implemented a rudimentary version of the first one by using a read only connection. You can see that this works as expected when we try to insert a new user into the database and get an error.

In [40]:
cursor = conn.cursor()

cursor.execute("INSERT INTO users (email, name) VALUES (?, ?)", ("test@test.com", "test"))
conn.commit()
conn.close()

OperationalError: attempt to write a readonly database

In [41]:
from pydantic import BaseModel
import instructor
import openai

client = instructor.from_openai(openai.AsyncOpenAI())


class SQLQuery(BaseModel):
    chain_of_thought: str
    query: str

    async def execute(self, conn: sqlite3.Connection):
        cursor = conn.cursor()
        try:
            print(f"Executing query: {self.query}")
            cursor.execute(self.query)
            return cursor.fetchall()
        except sqlite3.Error as e:
            print(f"Error executing query: {e}.")
            raise ValueError(f"Invalid SQL query: {e} when executing {self.query}")


schema = open("./init.sql", "r").read()


async def generate_and_execute_sql(
    client: openai.AsyncOpenAI,
    user_email: str,
    user_name: str,
    schema: str,
    user_query: str,
):
    resp = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""
                You are an expert text-2-sql model. You're about to be passed some user information, a database schema and a user query. Make sure you only write sql based off what's in the sqlite database schema defined below.

                Here is some information about the user
                - Email: {user_email}
                - Name: {user_name}

                Here is the database schema
                {schema}
                """,
            },
            {"role": "user", "content": user_query},
        ],
        response_model=SQLQuery,
    )
    return await resp.execute(conn)


query = "How much have I spent in 2023?"
query = await generate_and_execute_sql(client, email, name, schema, query)
print(query)

Executing query: SELECT SUM(total_amount) AS total_spent FROM orders WHERE user_email = 'allison_hill@gmail.com' 
AND date BETWEEN '2023-01-01' AND '2023-12-31';

[(2254.72,)]

In [42]:
query = "How many items have I bought?"
query = await generate_and_execute_sql(client, email, name, schema, query)
print(query)

Executing query: SELECT SUM(s.quantity) AS total_items_bought FROM sales s JOIN orders o ON s.order_id = o.order_id
WHERE o.user_email = 'allison_hill@gmail.com';

[(10,)]

## Integrating it into our current flow

Now that we've seen how we can use a language model to generate valid SQL queries, let's integrate it into our current flow.

We'll start by taking a function that will do the following

1. Given a user query, we'll extract out the relevant items from our product catalogue. This is done using the query understanding and filtering that we did in the previous notebook.
2. Next we'll take this data and get a model to determine if we need to make a second SQL call to fetch more data. If we do, we'll execute a SQL call based off the model's output.
3. Once we've done so, we'll generate a response using the retrieved data and the original user query.

It's important to note that this is a simplified version of what text-2-sql might look like in production.

In [1]:
from typing import Optional
from pydantic import BaseModel, model_validator
import openai
import json
import pandas as pd
import instructor


class Attribute(BaseModel):
    name: str
    values: list[str]


class QueryFilters(BaseModel):
    attributes: list[Attribute]
    material: Optional[list[str]]
    min_price: Optional[float] = None
    max_price: Optional[float] = None
    subcategory: str
    category: str
    product_type: list[str]
    occasions: list[str]

    @model_validator(mode="after")
    def validate_attributes(self):
        # Validate category exists in taxonomy
        if self.category not in taxonomy_data["taxonomy_map"]:
            raise ValueError(
                f"Invalid category: {self.category}. Valid categories are {taxonomy_data['taxonomy_map'].keys()}"
            )

        # Validate subcategory exists under category
        if self.subcategory not in taxonomy_data["taxonomy_map"][self.category]:
            raise ValueError(
                f"Invalid subcategory {self.subcategory} for category {self.category}. Valid subcategories are {taxonomy_data['taxonomy_map'][self.category]}"
            )

        # Validate product types
        valid_types = taxonomy_data["taxonomy_map"][self.category][self.subcategory][
            "product_type"
        ]
        for product_type in self.product_type:
            if product_type not in valid_types:
                raise ValueError(
                    f"Invalid product type: {product_type}. Valid product types are {valid_types}"
                )

        # Validate attributes
        valid_attrs = taxonomy_data["taxonomy_map"][self.category][self.subcategory][
            "attributes"
        ]
        for attr in self.attributes:
            if attr.name not in valid_attrs:
                raise ValueError(f"Invalid attribute name: {attr.name}")
            for value in attr.values:
                if value not in valid_attrs[attr.name]:
                    raise ValueError(
                        f"Invalid value {value} for attribute {attr.name}. Valid values are {valid_attrs[attr.name]}"
                    )

        # Validate occasions
        for occasion in self.occasions:
            if occasion not in taxonomy_data["occasions"]:
                raise ValueError(
                    f"Invalid occasion: {occasion}. Valid Occasions are {taxonomy_data['occasions']}"
                )

        # Validate materials if provided
        if self.material:
            for material in self.material:
                if material not in taxonomy_data["materials"]:
                    raise ValueError(
                        f"Invalid material: {material}. Valid Materials are {taxonomy_data['materials']}"
                    )

        return self


async def extract_query_filters(
    query: str, client: openai.AsyncOpenAI, taxonomy_data: dict
) -> QueryFilters:
    return await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """
                    You are a helpful assistant that extracts user requirements from a query.
                    
                    Use these references:
                    - Taxonomy: {{ taxonomy }}
                    - Valid occasions: {{ occasions }} 
                    - Valid materials: {{ materials }}
                    
                    Guidelines:
                    - If a filter isn't needed, return an empty list
                    - Only use values from the provided taxonomy, occasions and materials lists. If the attribute exists on multiple types, make sure that you only look at the specific types listed under the subcategory you have chosen
                    - if the user gives a range (Eg. around 50), just give a buffer of 20 on each side (Eg. 30-70)
                    - if the user gives a vague price (Eg. I have a high budget), just set max price to 1000
                    - Only add attributes and filters that a user has mentioned explicitly
                    - Cap sleeves are the same as sleeveless
                    - Only include a regular or relaxed fit if the user has explicitly mentioned it. If you do include either of them, include both in the filters
                    - only classify an item as unisex if the user has explicitly mentioned it. 

                    Extract the requirements and format them according to the QueryFilters model.
                """,
            },
            {"role": "user", "content": query},
        ],
        context={
            "taxonomy_map": taxonomy_data["taxonomy_map"],
            "taxonomy": taxonomy_data["taxonomy"],
            "occasions": taxonomy_data["occasions"],
            "materials": taxonomy_data["materials"],
        },
        response_model=QueryFilters,
    )


def retrieve_and_filter(query: str, table, filters: QueryFilters, max_k=100):
    query_parts = []

    # We do a prefilter on category,price and material since these will always be provided
    query_parts.append(f"category='{filters.category}'")
    query_parts.append(f"subcategory='{filters.subcategory}'")

    if filters.min_price:
        query_parts.append(f"price >= {filters.min_price}")
    if filters.max_price:
        query_parts.append(f"price <= {filters.max_price}")

    query_string = " AND ".join(query_parts)
    items = (
        table.search(query=query)
        .where(query_string, prefilter=True)
        .limit(max_k)
        .to_list()
    )

    items = [
        {
            **item,
            "attributes": json.loads(item["attributes"]),
            "occasions": json.loads(item["occasions"]),
        }
        for item in items
    ]

    if filters.product_type:
        items = [item for item in items if item["product_type"] in filters.product_type]

    if filters.material:
        items = [item for item in items if item["material"] in filters.material]

    if filters.occasions:
        items = [
            item
            for item in items
            if any(occasion in item["occasions"] for occasion in filters.occasions)
        ]

    if filters.attributes:
        for attr in filters.attributes:
            if not attr.values:
                continue
            curr_items = []
            for item in items:
                attr_name = attr.name
                attr_values = attr.values
                item_attr_values = item["attributes"]
                for item_attr in item_attr_values:
                    if (
                        item_attr["name"] == attr_name
                        and item_attr["value"] in attr_values
                    ):
                        curr_items.append(item)
                        break

            items = curr_items

    res = []

    for item in items:
        del item["_distance"]
        del item["vector"]
        res.append(item)

    return res


In [2]:
from helpers import process_taxonomy_file
import lancedb

taxonomy_data = process_taxonomy_file("./taxonomy.yml")
client = instructor.from_openai(openai.AsyncOpenAI())
db = lancedb.connect("./lancedb")
labelled_table = db.open_table("labelled_items")

query = "What are some good tank tops under 40, what sizes do you have it in?"
filters = await extract_query_filters(query, client, taxonomy_data)
items = retrieve_and_filter(query, labelled_table, filters)
pd.DataFrame(items)


/Users/ivanleo/Documents/coding/systematically-improving-rag/cohort_2/week5/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,title,description,brand,category,subcategory,product_type,attributes,occasions,material,pattern,price
0,87,Lace Trim Tank Top,Elevate your wardrobe with this elegant sleeve...,Zara,Women,Tops,Tank Tops,"[{'name': 'Sleeve Length', 'value': 'Sleeveles...","[Everyday Wear, Casual Outings, Smart Casual, ...",Cotton,Solid,28.53
1,183,Tie-Dye Sleeveless Top,Add a splash of color to your wardrobe with th...,Urban Outfitters,Women,Tops,Tank Tops,"[{'name': 'Sleeve Length', 'value': 'Sleeveles...","[Casual Outings, Everyday Wear, Smart Casual, ...",Cotton,Geometric,36.26


Now let's see how we can use this list of retrieved items to generate a SQL call if needed. I just want to point out two things here that are important in the response object

1. **Optional Field** - We want to use a `Optional` field here for the generated SQL call so that the model can decide if a SQL call is needed or not.

2. **RelevantItems** - We've also defined a `Item` model here for the model to use to denote what it identified as relevant. T

This is useful for two reasons

1. It allows us to monitor and audit what items the model is identifying as relevant. This is useful for debugging and improving the prompts and taxonomies we have in place.

2. By conditioning the final generated SQL on the list of relevant items, it increases the likelihood of the SQL being correct. 

In [16]:
from sqlite3 import Connection
from pydantic import Field, BaseModel
import openai
from typing import Optional


class Item(BaseModel):
    product_id: int
    title: str
    brand: str
    price:str
    material:str


class UserInformation(BaseModel):
    email: str
    name: str


class Response(BaseModel):
    response: str


class WriteSQL(BaseModel):
    relevant_items: list[Item]
    sql_query: str = Field(description="SQL query to be executed to answer the user's query")

    async def execute(
        self,
        conn: Connection,
    ):
        cursor = conn.cursor()
        cursor.execute(self.sql_query)
        results = cursor.fetchall()
        return [item for item in results]


class UserResponse(BaseModel):
    chain_of_thought: str
    use_sql: bool
    action: Optional[WriteSQL] = Field(
        description="If we need to make a SQL query, use this tool, else set to None"
    )

    async def execute(
        self,
        conn: Connection
    ):
        if self.use_sql:
            return None

        return await self.action.execute(conn)


Let's see how this works with our original query understanding portion


In [28]:
from rich import print
from helpers import process_taxonomy_file
import instructor
import lancedb

# Initialize dependencies
DB_PATH = "./db.sqlite"
TAXONOMY_PATH = "./taxonomy.yml"
LANCEDB_PATH = "./lancedb"

taxonomy_data = process_taxonomy_file(TAXONOMY_PATH)
client = instructor.from_openai(openai.AsyncOpenAI())
db = lancedb.connect(LANCEDB_PATH)
labelled_table = db.open_table("labelled_items")

# Load schema
with open("init.sql") as f:
    schema = f.read()

# Process query
query = "What are some good tank tops under 40? What sizes do you have it in?"

# Extract filters and retrieve items
filters = await extract_query_filters(query, client, taxonomy_data)
items = retrieve_and_filter(query, labelled_table, filters)

In [29]:
pd.DataFrame(items)

,id,title,description,brand,category,subcategory,product_type,attributes,occasions,material,pattern,price
0,87,Lace Trim Tank Top,Elevate your wardrobe with this elegant sleeve...,Zara,Women,Tops,Tank Tops,"[{'name': 'Sleeve Length', 'value': 'Sleeveles...","[Everyday Wear, Casual Outings, Smart Casual, ...",Cotton,Solid,28.53
1,183,Tie-Dye Sleeveless Top,Add a splash of color to your wardrobe with th...,Urban Outfitters,Women,Tops,Tank Tops,"[{'name': 'Sleeve Length', 'value': 'Sleeveles...","[Casual Outings, Everyday Wear, Smart Casual, ...",Cotton,Geometric,36.26


In [14]:
import sqlite3

conn = sqlite3.connect(DB_PATH)
info = UserInformation(email=email, name=name)


async def generate_response(query, conn, client, items, info, schema):
    return await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"""
You are an expert assistant that generates responses to user queries. We have identified some items in our product catalogue that are releevant to the user's query. 

Determine if further analysis is needed to answer the user' query of {query}. If so, write explicitly what you're going to do in the task field (including the fields you'd like to filter by and the data that you want to retrieve).

The user's information is: {info.model_dump()}

Here are the relevant items: {items}

Our Databse Schema is {schema}

Rules
- Price filters are already applied, there's no need to filter by price in the sql
- `id` is the product_id field in the database
            """,
            }
        ],
        response_model=UserResponse,
    )


resp = await generate_response(query, conn, client, items, info, schema)
print(resp)

UserResponse(
    chain_of_thought="The user's query about the tank tops involves knowing the sizes available for the items 
listed under $40. The product data provides details like id, title, brand, and price but does not include size, 
which is stored in the stock table. Therefore, additional information from the stock table is needed to identify 
available sizes.",
    use_sql=True,
    action=WriteSQL(
        relevant_items=[
            Item(product_id=87, title='Lace Trim Tank Top', brand='Zara', price='28.53', material='Cotton'),
            Item(
                product_id=183,
                title='Tie-Dye Sleeveless Top',
                brand='Urban Outfitters',
                price='36.26',
                material='Cotton'
            )
        ],
        sql_query='SELECT product_id, size FROM stock WHERE product_id IN (87, 183) AND quantity > 0'
    )
)

In [19]:
sql_data = await resp.execute(conn)
print(sql_data[:3])

[(87, 'XXL'), (87, 'XXS'), (87, 'XS')]

We can also validate that our model is able to inteligently decide whether it needs to make a SQL call or not.

For example if the user is just asking about specific items in the catalogue, then we don't need to make a SQL call as you can see below with `use_sql` being set to false.

In [31]:

resp = await generate_response("What are some good tank tops under 40?", conn, client, items, info, schema)
print(resp)

UserResponse(
    chain_of_thought="The user asked for tank tops under $40. We have a list of relevant items with their prices 
listed, and all of these items are under $40, as per the user's request. Therefore, there is no need for additional
analysis. I'll provide the user with the relevant details of these items.",
    use_sql=False,
    action=None
)

Now that we've fetched the relevant items based off the user's query, let's generate a response using the retrieved data and the original user query.

In [30]:
from rich import print

class FinalResponse(BaseModel):
    response: str


user_message = await client.chat.completions.create(
    response_model=FinalResponse,
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": """
            You are an expert assistant that generates a response to a user's query.

            The user's query was {{ query }}

            These were the relevant items in our catalogue
            {% for item in items %}
            - {{ item }}
            {% endfor %}

            {% if retrieved_items is not none %}
            This is what we found when we ran the SQL query - {{ sql_query }}
                {% for item in retrieved_items %}
                - {{ item }}
                {% endfor %}
            {% endif %}

            Rules
            - When responding to the user, make sure to use the product titles and brand from the relevant items
            - If the user asks about a specific item, make sure to mention the item title and brand in your response, do not leak product_id
            """
        }
    ],
    context={
        "query": query,
        "items": items,
        "retrieved_items": sql_data,
        "sql_query": resp.action.sql_query,
    },
)
print(user_message.response)

Here are two great tank tops under $40 from our catalog:

1. **Lace Trim Tank Top** by Zara
   - Price: $28.53
   - Description: Elevate your wardrobe with this elegant sleeveless tank top, featuring a delicate lace trim and a
flattering V-neck design. It's perfect for layering or wearing alone, offering both sophistication and versatility.
   - Available Sizes: XXS, XS, M, L, XXL

2. **Tie-Dye Sleeveless Top** by Urban Outfitters
   - Price: $36.26
   - Description: Add a splash of color to your wardrobe with this vibrant tie-dye sleeveless top. Its relaxed fit 
and flowing hem offer comfort with a trendy look, perfect for casual outings.
   - Available Sizes: XXL

These tops are stylish and affordable, available in a variety of sizes to suit your needs.

In this notebook, we've seen how we can use function calling to generate valid SQL queries. We've also seen how we can use function calling to allow the model to selectively generate SQL queries when needed.

In a production setting, you'd probably also want to consider the following
- Using more robust safeguards so that the model doesn't run arbitrary SQL queries
- Adding in checks to ensure the items we're passing into the prompt doesn't overflow the context window

More importantly, as we deploy these systems in production, it might make sense to start developing specialised tooling for the specific tasks (Eg. always returning the available sizes and colors for a given product when we fetch it or adding the ability for a language model to trigger an order on behalf of the customer) based on what the user is asking for.

In next week's notebook, we'll look into how we can implement more complex tooling for our language model to perform external tasks and how we can deal with multiple tools being called in a single prompt.
